# Step1 - Set the ZIPLINE_ROOT

In [7]:
import os


os.environ['ZIPLINE_ROOT'] = os.path.join(os.getcwd(), '.zipline')

os.listdir(os.environ['ZIPLINE_ROOT'])


os.environ['ZIPLINE_TRADER_CONFIG'] = os.path.join(os.getcwd(), "./zipline-trader.yaml")

with open(os.environ['ZIPLINE_TRADER_CONFIG'], 'r') as f:
    data = f.read()
    print(data[:20])

['custom_data', 'data', 'extension.py']

# Step2 - Load your Bundle

In [10]:
import zipline
from zipline.data import bundles

bundle_name = 'alpaca_api'
bundle_data = bundles.load(bundle_name)

# Step3 - Create The Data Portal

In [25]:
from zipline.pipeline.loaders import USEquityPricingLoader
from zipline.utils.calendars import get_calendar
from zipline.pipeline.data import USEquityPricing
from zipline.data.data_portal import DataPortal

import pandas as pd

# Set the dataloader
pricing_loader = USEquityPricingLoader.without_fx(bundle_data.equity_daily_bar_reader, bundle_data.adjustment_reader)


# Define the function for the get_loader parameter
def choose_loader(column):
    if column not in USEquityPricing.columns:
        raise Exception('Column not in USEquityPricing')
    return pricing_loader

# Set the trading calendar
trading_calendar = get_calendar('NYSE')

start_date = pd.Timestamp('2019-07-05', tz='utc')
end_date = pd.Timestamp('2020-11-13', tz='utc')

# Create a data portal
data_portal = DataPortal(bundle_data.asset_finder,
                         trading_calendar = trading_calendar,
                         first_trading_day = start_date,
                         equity_daily_reader = bundle_data.equity_daily_bar_reader,
                         adjustment_reader = bundle_data.adjustment_reader)


#  Let's Get Some Historical Data

In [26]:
equity = bundle_data.asset_finder.lookup_symbol("ACES", end_date)
data_portal.get_history_window(assets=[equity], end_dt=end_date, bar_count=10,
                               frequency='1d',
                               field='close',
                               data_frequency='daily')

Equity(604 [ACES])
2020-11-02 00:00:00+00:00               57.10
2020-11-03 00:00:00+00:00               58.50
2020-11-04 00:00:00+00:00               57.16
2020-11-05 00:00:00+00:00               60.86
2020-11-06 00:00:00+00:00               61.23
2020-11-09 00:00:00+00:00               61.54
2020-11-10 00:00:00+00:00               60.25
2020-11-11 00:00:00+00:00               62.42
2020-11-12 00:00:00+00:00               61.77
2020-11-13 00:00:00+00:00               61.82